## RESTING STATE data: raw --> epochs
This script uses the hand cleaned raw data and creates fixed length epochs based on this data. Segments that have been marked as bad are excluded from the epoch generation. 

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from tqdm import tqdm 
from atpbar import atpbar
from datetime import datetime
import mne
import autoreject
from autoreject import AutoReject
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from pyprep.find_noisy_channels import NoisyChannels
from mne_icalabel import label_components
import re



In [6]:
# Suppress MNE output
mne.set_log_level('WARNING')


## Make fixed length epochs

In [34]:
# import data from Jhana subjects
paths = glob.glob(f"../hand_cleaning/resting_state/*")
paths.sort()
for path in paths: 
    raw = mne.io.read_raw_fif(path, preload=True)
    #raw.filter(0.3, 45)
    epochs = mne.make_fixed_length_epochs(raw, duration=3, overlap=0, preload=True, reject_by_annotation=True)

    #SAVING
    #basename = os.path.basename(path)[:-8]
    components = re.split('[-_]', os.path.basename(path))
    export_name = f'sub{components[0][3:].zfill(2)}-{components[1]}-{components[2]}_epo.fif'
    epochs.save(f'epochs/{export_name}', overwrite=False)

# Import the control data 
paths = glob.glob(f"../hand_cleaning/control_resting_state/*")
paths.sort()
for path in paths: 
    raw = mne.io.read_raw_fif(path, preload=True)
    #raw.filter(0.3, 45)
    epochs = mne.make_fixed_length_epochs(raw, duration=3, overlap=0, preload=True, reject_by_annotation=True)

    #SAVING   
    components = re.split('[-_]', os.path.basename(path))
    sub_n = components[0][3:]
    export_name = f'con{sub_n.zfill(2)}-day0-control_epo.fif'
    epochs.save(f'epochs/{export_name}', overwrite=False)

Opening raw data file ../hand_cleaning/resting_state/sub0-day1-jhana-raw.fif...
    Range : 512 ... 388096 =      2.000 ...  1516.000 secs
Ready.
Reading 0 ... 387584  =      0.000 ...  1514.000 secs...
Not setting metadata
504 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 504 events and 768 original time points ...
98 bad epochs dropped
Opening raw data file ../hand_cleaning/resting_state/sub0-day1-mindfulness-raw.fif...
    Range : 512 ... 384768 =      2.000 ...  1503.000 secs
Ready.
Reading 0 ... 384256  =      0.000 ...  1501.000 secs...
Not setting metadata
500 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 500 events and 768 original time points ...
56 bad epochs dropped
Opening raw data file ../hand_cleaning/resting_state/sub0-day2-jhana-raw.fif...
    Range : 512 ... 384768 =      2.000 ...  1503.000 secs
Ready.
Reading 0 ... 384256  =     

## Autoreject fixed length epochs

In [35]:
paths = glob.glob('epochs/con*')
paths.sort()
for path in paths: 
    basename = os.path.basename(path)
    epochs = mne.read_epochs(path)
    ar = AutoReject()
    epochs_clean = ar.fit_transform(epochs)
    epochs_clean.save(f'epochs_ar/{basename}', overwrite=False)

Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/resting state/epochs/con00-day0-control_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2996.09 ms
        0 CTF compensation matrices available
Not setting metadata
356 matching events found
No baseline correction applied
0 projection items activated
Running autoreject on ch_type=eeg


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 7 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


  0%|          | Creating augmented epochs : 0/25 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/25 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/356 [00:00<?,       ?it/s]

...

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.90 and n_interpolate=4


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 9 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


  0%|          | Repairing epochs : 0/438 [00:00<?,       ?it/s]

No bad epochs were found for your data. Returning a copy of the data you wanted to clean. Interpolation may have been done.


## Epochs based on Jhana

### buttons_to_events function

In [38]:
def buttons_to_events(sd_array, threshold, plot_button_events, summing_channels=True, title='none'):
    """
    this function takes a numpy array containing the button presses and outputs two lists,
    one containing the timepoints of events, the other containing the label
    """
    running = True
    i = 0
    markers_loc = []
    markers_val = []
    value = 1

    if summing_channels == True:
        # if no button is pressed
        if sum(sum(sd_array[:, 3:])) == 0:
            return markers_loc, markers_val

        buttons = np.sum(sd_array[:, 3:], axis=1)
        button_loc = np.where(buttons[:-1] < buttons[1:])[0]
        # if button is pressed only once
        if len(button_loc) == 1:
            markers_loc.append(button_loc[i])
            markers_val.append(value)
            return markers_loc, markers_val

    if summing_channels == False:
        if np.sum(sd_array) == 0:
            return markers_loc, markers_val
            print("no events found")
        buttons = sd_array
        button_loc = np.where(buttons[:-1] < buttons[1:])[0]
    value = 1
    while running == True:
        if len(button_loc) == 1:
            markers_loc.append(button_loc[i])
            markers_val.append(1)
            running = False
            i = i + 1
        elif button_loc[i + 1] - button_loc[i] > threshold:
            markers_loc.append(button_loc[i])
            markers_val.append(value)
            value = 1
            i = i + 1
        elif button_loc[i + 1] - button_loc[i] < 64:  # min threshold
            i = i + 1
        else:
            value = value + 1
            i = i + 1

        if len(button_loc) == i + 1:
            markers_loc.append(button_loc[i])
            markers_val.append(value)
            running = False

#    if markers_val.count(1) == 2 and markers_val[0] == 1 and markers_val[1] == 1:
#        markers_val[0] = 0

    if plot_button_events == True:
        plt.figure()
        plt.plot(buttons)
        for value, xc in zip(markers_val, markers_loc):
            plt.axvline(x=xc, label="line at x = {}".format(value), c="red")
            plt.text(xc + 1000, 0.9, value, {"size": 15})
        plt.title(title)
        plt.show()
    return markers_loc, markers_val

### Actual code

In [36]:
counter = 0

In [39]:
plt.close('all')
paths = glob.glob(f"../hand_cleaning/resting_state/*jhana-raw.fif") #exclude mindfulness clicks (only 2 anyway)
paths.sort()
run = True
while run:
    path = paths[counter]
    basename = os.path.basename(path)[:-8]
    raw = mne.io.read_raw_fif(path, preload=True)
    buttons_raw = (raw[33][0][0] + raw[34][0][0] + raw[35][0][0])
    button_sum = np.sum(buttons_raw)
    if button_sum != 0:
        
        markers_loc, markers_val = buttons_to_events(
            sd_array=buttons_raw, threshold=1000, plot_button_events=False, summing_channels=False, title=basename
        )
        
        end_time = markers_loc[0]/256
        raw_cropped = raw.copy().crop(0, end_time)
        epochs = mne.make_fixed_length_epochs(raw_cropped, duration=3, overlap=0, preload=True, reject_by_annotation=True)
        #epochs.save(f'epochs_button/{basename}_button#B_jhanaB_epo.fif', overwrite=True)
        
        for i, val in enumerate(markers_val):
            start_time = markers_loc[i] / 256
            if len(markers_loc) == i + 1:
                end_time = None
            else:
                end_time = markers_loc[i + 1] / 256
            raw_cropped = raw.copy().crop(start_time, end_time)
            epochs = mne.make_fixed_length_epochs(raw_cropped, duration=3, overlap=0, preload=True, reject_by_annotation=True)
            #epochs.save(f'epochs_button/{basename}_button#{i}_jhana{markers_val[i]}_epo.fif', overwrite=True)
        
        print (counter)
        counter+=1
        run = False
    else:
        counter+=1
        


Opening raw data file ../hand_cleaning/resting_state/sub1-day1-jhana-raw.fif...
    Range : 768 ... 388352 =      3.000 ...  1517.000 secs
Ready.
Reading 0 ... 387584  =      0.000 ...  1514.000 secs...
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 768 original time points ...
5 bad epochs dropped
Not setting metadata
9 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 9 events and 768 original time points ...
5 bad epochs dropped
Not setting metadata
67 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 67 events and 768 original time points ...
10 bad epochs dropped
Not setting metadata
109 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 109 events and 768 original time points ...
3

## AR on Jhana epochs

In [245]:
paths = glob.glob('epochs_button/*')
paths.sort()
for path in paths: 
    basename = os.path.basename(path)
    export_path = f'epochs_button_ar/{basename}'
    if not os.path.exists(export_path):
        epochs = mne.read_epochs(path)
        if len(epochs)>=10:
            ar = AutoReject()
            epochs_clean = ar.fit_transform(epochs)
            epochs_clean.save(export_path, overwrite=True)

    else:
        print(f'already done: export_path')

already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done: export_path
already done

/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_6335/2509468502.py:7: RuntimeWarning: epochs._get_data() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.
  epochs = mne.read_epochs(path)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_6335/2509468502.py:7: RuntimeWarning: epochs._get_data() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.
  epochs = mne.read_epochs(path)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_6335/2509468502.py:7: RuntimeWarning: epochs._get_data() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.
  epochs = mne.read_epochs(path)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_6335/2509468502.py:7: RuntimeWarning: epochs._get_data() ca